In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

print("Reading data")
print(os.listdir("../NLPStuff"))

##For data preprocessing
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

##For Machine Learning
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
# Any results you write to the current directory are saved as output.

Reading data
['folksonomication_source-master', 'mpst_full_data.csv', '.ipynb_checkpoints', 'CATS-Main.ipynb', 'partition.json', 'MPSTGenreCorrected.csv', 'GenreCorrected.csv', 'wiki_movie_plots_deduped.csv', 'Untitled.ipynb']


In [20]:
df1 = pd.read_csv("../NLPStuff/wiki_movie_plots_deduped.csv")
df1.tail()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
Release Year        34886 non-null int64
Title               34886 non-null object
Origin/Ethnicity    34886 non-null object
Director            34886 non-null object
Cast                33464 non-null object
Genre               34886 non-null object
Wiki Page           34886 non-null object
Plot                34886 non-null object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


In [21]:
df2 = pd.read_csv("../NLPStuff/mpst_full_data.csv")
df2.tail()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14828 entries, 0 to 14827
Data columns (total 6 columns):
imdb_id            14828 non-null object
title              14828 non-null object
plot_synopsis      14828 non-null object
tags               14828 non-null object
split              14828 non-null object
synopsis_source    14828 non-null object
dtypes: object(6)
memory usage: 695.1+ KB


In [23]:
df1['Genre']=df1['Genre'].replace('unknown',np.nan)
df1=df1.dropna(axis=0, subset=['Genre'])
print(df1.info())
df2['tags']=df2['tags'].replace('unknown',np.nan)
df2=df2.dropna(axis=0, subset=['tags'])
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28803 entries, 6 to 34885
Data columns (total 8 columns):
Release Year        28803 non-null int64
Title               28803 non-null object
Origin/Ethnicity    28803 non-null object
Director            28803 non-null object
Cast                28064 non-null object
Genre               28803 non-null object
Wiki Page           28803 non-null object
Plot                28803 non-null object
dtypes: int64(1), object(7)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14828 entries, 0 to 14827
Data columns (total 6 columns):
imdb_id            14828 non-null object
title              14828 non-null object
plot_synopsis      14828 non-null object
tags               14828 non-null object
split              14828 non-null object
synopsis_source    14828 non-null object
dtypes: object(6)
memory usage: 810.9+ KB
None


In [24]:
print(len(df1))
print(df1.shape)
a=df1['Genre'].value_counts()[:500]
b=a.keys().tolist()
print(b)
df1=df1[df1.Genre.isin(b)]
df1=df1.reset_index(drop=True)
print(len(df2))
print(df2.shape)
c=df2['tags'].value_counts()[:2000]
d=c.keys().tolist()
print(d)
df2=df2[df2.tags.isin(d)]
df2=df2.reset_index(drop=True)

28803
(28803, 8)
['drama', 'comedy', 'horror', 'action', 'thriller', 'romance', 'western', 'crime', 'adventure', 'musical', 'crime drama', 'romantic comedy', 'science fiction', 'film noir', 'mystery', 'war', 'animation', 'comedy, drama', 'sci-fi', 'family', 'fantasy', 'animated', 'musical comedy', 'comedy-drama', 'biography', 'anime', 'suspense', 'romantic drama', 'comedy drama', 'animated short', 'drama, romance', 'social', 'historical', 'documentary', 'action thriller', 'serial', 'world war ii', 'family drama', 'war drama', 'drama, crime', 'comedy, musical', 'comedy/drama', 'comedy, romance', 'romance, drama', 'biopic', 'crime thriller', 'historical drama', 'black comedy', 'action comedy', 'comedy short', 'superhero', 'crime comedy', 'horror comedy', 'martial arts', 'action, drama', 'drama, biography', 'action, romance', 'drama, war', 'action, thriller', 'romance/comedy', 'social drama', 'drama, adventure', 'melodrama', 'romance/drama', 'action, comedy', 'action drama', 'biography, d

In [27]:
def plotToWords(raw_plot):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_plot)
    lower_case = letters_only.lower()
    words = lower_case.split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops]
    return (" ".join(meaningful_words))

def preprocess(dataframe):
    clean_train_reviews = []
    for i in range(0,len(dataframe)):
        clean_train_reviews.append(plotToWords(dataframe.iloc[i]['Plot']))
    dataframe['Plot']=clean_train_reviews
    return dataframe
def preprocess2(dataframe):
    clean_train_reviews = []
    for i in range(0,len(dataframe)):
        clean_train_reviews.append(plotToWords(dataframe.iloc[i]['plot_synopsis']))
    dataframe['plot_synopsis']=clean_train_reviews
    return dataframe

df1=preprocess(df1)
print(df1["Plot"][:10])
df2=preprocess2(df2)
print(df2["plot_synopsis"][:10])



0    film opens two bandits breaking railroad teleg...
1    film family move suburbs hoping quiet life thi...
2    rarebit fiend gorges welsh rarebit restaurant ...
3    irish villager kathleen tenant captain clearfi...
4    boone daughter befriends indian maiden boone c...
5    heading baseball game nearby ballpark sports f...
6    plot black woman going dentist toothache given...
7    beautiful summer day father mother take daught...
8    thug accosts girl leaves workplace man rescues...
9    young couple decides elope caught midst romant...
Name: Plot, dtype: object
0    two thousand years ago nhagruul foul sorcerer ...
1    matuschek gift store budapest workplace alfred...
2    george falconer colin firth approaches car acc...
3    small time italian american criminals caesar e...
4    movie begins video shot men hands tied behind ...
5    somafree institute dr hal raglan humiliates pa...
6    brothers terry colin farrell ian ewan mcgregor...
7    arrival train pulls station north 

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), max_features=10000)
features = tfidf.fit_transform(df1.Plot).toarray()
labels = df1.Genre
features.shape

(26825, 10000)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf2 = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), max_features=10000)
features2 = tfidf2.fit_transform(df2.plot_synopsis).toarray()
labels2 = df2.tags
features2.shape

(11224, 10000)

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df1['Plot'], df1['Genre'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [32]:
#VERY MEMORY / CPU INTENSIVE
#RUN ONLY ON A GOOD MACHINE!

#X_train2, X_test2, y_train2, y_test2 = train_test_split(df2['plot_synopsis'], df2['tags'], random_state = 0)
#count_vect2 = CountVectorizer()
#X_train_counts2 = count_vect2.fit_transform(X_train2)
#tfidf_transformer2 = TfidfTransformer()
#X_train_tfidf2 = tfidf_transformer2.fit_transform(X_train_counts2)
#clf2 = MultinomialNB().fit(X_train_tfidf2, y_train2)

In [146]:
elem = 6
test = [df2['plot_synopsis'][elem]]
actual = [df2['tags'][elem]] 
title = [df2['title'][elem]]
print('Best prediction for the primary tag is: ')
preds = np.argsort(clf.predict_proba(count_vect.transform(test)),axis=1)[:,-3:]
vals = [b[i] for i in preds.flatten()]
print(vals)
print('Best associated tags: ')
print(actual)
print('Title: ')
print(title)

Best prediction for the primary tag is: 
['crime, horror', 'family, drama', 'sexploitation']
Best associated tags: 
['tragedy, dramatic, murder']
Title: 
["Cassandra's Dream"]
